<a href="https://colab.research.google.com/github/TBFY/api-gateway/blob/master/notebooks/supplier_resource_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![TBFY-Banner](https://raw.githubusercontent.com/TBFY/general/master/figures/tbfy-banner.png)

# **List of awardees**
This notebook is an example of how to use a new resource created that shows a list of all the organisations that have been awarded. This resource is called "/supplier" and has been created in response to the needs of some business cases.

The resource has several fully customizable filters. Depending on the number of filters used, the response time may be affected.

In [0]:
import requests
import pandas as pd
from IPython.display import display, HTML

In [0]:
#@title Search Suppliers

nm = ""
jur = ""
stat = ""
tit = ""
des = ""
cur = ""
emp = ""
sdat = ""
edat = ""

name_org =  ""#@param {type:"string"}

jurisdiction = "es" #@param ["", "gb", "es", "fr", "it", "pt", "no", "si"] {type:"string"}   

min_employees =   0#@param {type:"integer"}

award_status = "" #@param ["", "pending", "active", "cancelled", "unsuccessful"] {type:"string"}   

award_title =  "" #@param {type:"string"}

award_description =  "" #@param {type:"string"}

currency = "" #@param {type:"string"}

start_date = "" #@param {type:"date"}

end_date = "" #@param {type:"date"}

max_number = 5 #@param {type:"slider", min:1, max:100, step:1}

if len(name_org) >0:
  nm = '&name=' + name_org
if len(jurisdiction)>0:
  jur = '&jurisdiction=' + jurisdiction 
if len(award_status)>0:
  stat = '&status=' + award_status 
if len(award_title)>0:
  tit = '&title=' + award_title 
if len(award_description)>0:
  des = '&description=' + award_description
if len(currency)>0:
  cur = '&currency=' + currency
if len(start_date)>0:
  sdat = '&start_dt=' + start_date + 'T00:00:00Z'
if len(end_date)>0:
  edat = '&end_dt=' + end_date + 'T00:00:00Z'
if min_employees>0:
  emp = '&minemployees=' + str(min_employees)

params = 'http://tbfy.librairy.linkeddata.es/api/supplier?size=' + str(max_number) + nm + jur + stat + tit + des + cur + emp + sdat + edat
print ('Requesting : ' + params )
print()
resp = requests.get(params)

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

df = pd.DataFrame(columns=['legalName', 'organisation', 'employees', 'award_id', 'award_title', 'award_description', 'award_date'])
i=0
for todo_item in resp.json():
  df.loc[i] = [todo_item['legalName'],todo_item['organisation'],todo_item['numberOfEmployees'],todo_item['award']['id'],todo_item['award']['title'],todo_item['award']['description'],todo_item['award']['date']]
  i+=1

display(HTML(df.to_html(justify='left')))

#Reconciliation

Once we have a list of awardees we can also match organisation names to legal corporate entities using the OpenRefine Reconciliation API

In [0]:
#@title Reconcile Suppliers

name_org =  ""#@param {type:"string"}
newname= name_org.replace(" ", "+")
jurisdiction = "es" #@param ["gb", "es", "fr", "it", "pt", "no"] {type:"string"}  

params = 'https://opencorporates.com/reconcile/' + jurisdiction + '?query=' + newname
resp = requests.get(params)

print ('Requesting : ' + params )
print ()
json_val = resp.json()['result']

if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

df = pd.DataFrame(columns=['id', 'name', 'score', 'match', 'uri'])
i=0
for todo_item in json_val:
  new_id = todo_item['id'].replace("/companies/","")
  new_id = new_id.replace("/","-")
  df.loc[i] = [new_id, todo_item['name'], todo_item['score'], todo_item['match'], todo_item['uri']]
  i+=1

display(HTML(df.to_html(justify='left')))
